# *Relatório - Projeto 2*

## *Algoritmos Genéticos*

### *Grupo:*
- João Paulo Soubihe RA 151106
- Matheus Silva Capeletti RA 203587
- Lucas Barretto Andrade RA 182371
- Karol Daniela Cala Pinzón RA 264308
- Johana Marisol Saravia Bulla RA 264319

### *Contribuições:*
- João: elaboração do problema e seu respectivo modelo de representação, roulette method
- Matheus: inicializador de população aleatória, seleção por torneio, solver, função de fitness, coleta de dados
- Lucas: mutação por flip, descrição do modelo utilizado.
- Karol: crossover, introdução
- Johana: mutação destrutiva, generativa e por swap


### *Introdução*

Devido ao problema do Covid-19, a população está de frente de uma possível superlotação de seus hospitais, faltando leitos para atender aos pacientes que já existiam e aos novos, vítimas da pandemia.

Aqui fazemos uma análise de como os hospitais estão localizados e como eles poderiam ser acomodados para melhor cobrir a região e, assim, poder atender os habitantes de maneira eficiente.

Em relação à falta de cobertura hospitalar, em certas áreas das cidades e que estão sendo bastante evidenciadas pela situação atual de pandemia, optou-se por realizar um algoritmo para realizar uma análise dos hospitais atual e também permite otimizar a localização dos hospitais de acordo com as necessidades da população, permitindo, no futuro, atender ao maior número possível de populações e melhorar o sistema de saúde, o que esse algoritmo faz é levar a localização dos hospital e atribuir a eles um intervalo, esse intervalo é usado para medir o perímetro em (km) em que o hospital é responsável por cuidar das pessoas, O objetivo do algoritmo é determinar a localização ideal para os hospitais, evitando que algumas áreas da cidade possuam vários hospitais, enquanto outras não, os hospitais não estariam aproveitando seu desempenho máximo.

### *Modelo*

O problema foi modelado da seguinte maneira:

Uma **cidade**, idealmente, é tratada como uma dimensão retangular (X, Y), em que X é a largura e Y o comprimento da cidade. Esta abstração pode ser elevada para um estado ou até mesmo para um país. Para se representar uma cidade real, pode-se fazer um mapeamento da região da cidade para um retângulo.

Cada **hospital** tem sua coordenada (X, Y) dentro da cidade, e também um número que determina sua capacidade em área coberta da cidade. Este número, nos casos de teste aqui são hipotéticos, mas podem ser calculados em função da densidade populacional da região, capacidade do hospital e suas particularidades.

O objetivo deste trabalho foi otimizar a área de de uma cidade que é atendida por pelo menos um hospital. Para isso foi utilizado um *algoritmo genético*.

#### Considerações para o *algoritmo genético*
Para se utilizar o algoritmo genético consideramos que um **indivíduo** (ou cromossomo) é um conjunto de hospitais da cidade, sendo que cada hospital é um **gene**.
As possíveis combinações de propriedades de um hospital (coordenadas e área de cobertura) formam o conjunto dos possíveis **alelos** de um gene.
**Mutações** de um indivíduo podem adicionar um hospital à cidade, remover um hospital da cidade, alterar sua posição ou área de cobertura, ou substituir os valores entre diferentes hospitais.
A **Reprodução** entre dois indivíduos acontece por _crossover_, selecionando aleatoriamente conjuntos compatíveis de genes de cada cromossomo, é gerado um conjunto de hospitais selecionando subconjuntos de dois indivíduos diferentes e fazendo a união deles.

O algoritmo funciona da seguinte maneira: primeiro, a população inicial é dada pela descrição da cidade na forma atual, ou pode ser criada aleatoriamente, cada indivíduo contém uma lista x de hospitais localizados na área da cidade, o algoritmo será interrompido quando o indivíduo que cobre na maior área possível e melhor localizada, se o algoritmo não for seguido, ele continuará sendo executado com variações das gerações e através da reprodução sexual e assexual.

### *Aptitude*



Com base no contexto escolhido precisamos escolher uma função que de certa forma avalie a cobertura que essa distribuição de hospitais poderiam trazer para a população.

Definimos que cada hospital teria em sua estrutura de dados um _range_. Tal atributo nos daria de certa forma a dimensão da unidade e o alcance que conseguiríamos abranger.

Supondo, por exemplo, que um hospital de range 2 está localizado em um espaço (x,y) do nosso grid. Esse centro poderá atender pessoas localizadas nos espaços

- (x,y)
- (x + 1, y)
- (x + 2, y)
- (x - 1, y)
- (x - 2, y)
- (x, y + 1)
- (x, y + 2)
- (x, y - 1)
- (x, y - 2)

<div style="text-align: center"><img src="range.png" width="350" /></div>


Outra decisão importante que tivemos que pensar é a qual penaliza uma certa solução. 

Como estamos trabalhando com indivíduos de tamanho variável, precisamos de algum mecanismo que garanta a veracidade de nossa simulação e de certa forma limite essa expansão. Aliás, se passarmos para o mundo real, cada hospital tem um custo para os governantes não é mesmo?

Pensando em uma simulação mais próxima da realidade e também pensando na diversidade entre os hospitais, impomos uma outra restrição a um range muito grande. Um hospital naturalmente não conseguirá atender um raio muito acima do que o normal. Assim procuramos de certa maneira normalizar esse range através da razão pelo máximo alcance de um indivíduo dessa população.
 
Quantizamos esse aspecto na forma de um ponto a menos em nossa avaliação para cada hospital compondo a solução.

Assim, com os parâmetros descritos, podemos quantizar a capacidade de atendimento de cada população. Através da equação:

```python
    fitness = (area - custo) / maior_alcance
```

Onde:

 - _area_ é dada pela quantidade de espaços sendo englobados no alcance de todos os hospitais em conjunto, *sem contabillizar duplicatas*;
 - _custo_ é dado pela soma das penalidades de cada solução, ou seja, o tamanho delas, ou o número de hospitais sendo considerados.

```python
@memoize_fitness
def fitness_function1(individual):
    #Cada hospital gera uma penalidade de 1 ponto
    #O maior alcance dos hospitais também gera uma penalidade
    #O valor da aptidão será (area - custo) / max(range)
    reached = []
    custo = len(individual)
    maxRange = 1
    for hospital in individual:
        reached.extend(hospital.get_reach())
        if hospital.range > maxRange:
            maxRange = hospital.range

    area = len(set(reached))
    fitness = float(area - custo) / maxRange
    return fitness
```



### *Dados*

O algoritmo se comportou de maneiras bem distintas alterando-se entre seleção por roleta ou torneio, outros fatores relevantes (que não sejam tamanho da população, número de iterações ou critério de convergência) na qualidade da solução são as taxas de mutação e crossover, mutações geram soluções melhores com mais frequência do que crossover.

Um ponto interessante é que a seleção por roleta requer um critério de convergência alto, pois suas soluções oscilam bastante, um critério pequeno nas primeiras execuções resultou no algoritmo parando a execução antes do limite de iterações, porém seu tempo de execução para um tamanho de população e limite de iterações é melhor que a seleção por torneio.
Seguem imagens de alguns cenários de execução (todos para uma cidade de dimensões 100x100 e inicialização semelhante):


<div style="text-align: center"><img src="Progression1.png" width="500" /></div>

<div style="text-align: center"><img src="Progression6.png" width="500" /></div>

<div style="text-align: center"><img src="Progression2.png" width="500" /></div>

<div style="text-align: center"><img src="Progression7.png" width="500" /></div>

<div style="text-align: center"><img src="Progression3.png" width="500" /></div>

<div style="text-align: center"><img src="Progression8.png" width="500" /></div>

<div style="text-align: center"><img src="Progression4.png" width="500" /></div>

<div style="text-align: center"><img src="Progression9.png" width="500" /></div>

<div style="text-align: center"><img src="Progression5.png" width="500" /></div>

<div style="text-align: center"><img src="Progression10.png" width="500" /></div>

Seja _m_ o tamanho máximo da população, _k_ o número máximo de iterações, _t_ o tamanho do torneio o tempo de execução do algoritmo é da ordem __O__(_m_*_k_*_t_), para a seleção por roleta tome _t = 1_. A memória consumida é da ordem __O__(_m_).

### *Particularidades e restrições*

Como todo o modelo, adaptações que abstraissem a realidade tiveram que ser feitas e, por consequência, obrigou-nos a considerar certas restrições.

Um dos fatores mais relevantes dentre os quais não foram considerados foi o da concentração populacional. Por exemplo, certas regiões podem necessitar de mais hospitais devido ao grande número de habitantes ali sediados. Para o nosso modelo considerou-se uma concentração uniforme de pessoas dentre o grid de representação regional.

Esse grid considerado em nossa implementação também é um importante fator a ser comentado. Por conveniência, de forma a simplificar o problema e possibilitar uma exploração lógica do mesmo, consideramos que a região avaliada seria formada por um grid de coordenadas (x,y). Cada unidade desse espaço pode ser considerada como uma combinação _latitude,longitude_ da região.

Dessa maneira uma importante questão foi levantada. Como poderíamos representar a capacidade de atendimento de um hospital, a fim de avaliarmos se a solução atendia uma maior ou menor área dentro de nosso grid.

Pelo aspecto indivisível das posições, abstraimos uma espécie de _range de atendimento_ de cada unidade médica, como um raio de cobertura. Imprecisões quanto a esse dado podem ser qualificadas em uma eventual validação do modelo, mas consideramos que a maneira abordada descreveu bem, diante das limitações da representação, um possível alcance.